In [15]:
import scipy
from scipy.stats import gamma
import numpy as np
import pandas as pd
import plotly
import plotly.express as px

from src.incoming_wip.incoming_wip_generator import IncomingWipGenerator
from src.incoming_wip.incoming_wip_config import default_incoming_wip_config
from src.dp_model.dp_model_config import default_dp_model_config
from src.dp_model.dp_model import DpModel

In [ ]:

x_values = np.linspace(0, 10, 100)
y_values = gamma.pdf(x=x_values, a=3, scale=1, loc=5)
gamma_df = pd.DataFrame({'x': x_values, 'y': y_values})

fig = px.scatter(data_frame=gamma_df, x='x', y='y')
fig.show()

In [ ]:
incoming_wip = IncomingWipGenerator(default_incoming_wip_config).generate()
plot_incoming_wip_df = pd.DataFrame({'incoming_wip': incoming_wip, 'period': range(len(incoming_wip))})
fig = px.line(plot_incoming_wip_df, x='period', y='incoming_wip')
fig.show()

In [ ]:
dp_model = DpModel(default_dp_model_config)
dp_model.run(incoming_wip)
dp_model.get_best_end_state()

In [20]:
state_df = dp_model.get_best_state_df()
state_df

,period,active_tank,current_slurry_unit_0,cleaning_period_remaining_0,filling_period_remaining_0,down_start_period_0,current_slurry_unit_1,cleaning_period_remaining_1,filling_period_remaining_1,down_start_period_1,current_slurry_unit_2,cleaning_period_remaining_2,filling_period_remaining_2,down_start_period_2,cost
0,0,0,200,0,0,inf,200,0,0,inf,200,0,0,inf,0
1,1,0,200,0,0,inf,200,0,0,inf,200,0,0,inf,0
2,2,0,200,0,0,inf,200,0,0,inf,200,0,0,inf,0
3,3,0,200,0,0,inf,200,0,0,inf,200,0,0,inf,0
4,4,0,200,0,0,inf,200,0,0,inf,200,0,0,inf,0
5,5,0,200,0,0,inf,200,0,0,inf,200,0,0,inf,0
6,6,0,199,0,0,inf,200,0,0,inf,200,0,0,inf,0
7,7,0,197,0,0,inf,200,0,0,inf,200,0,0,inf,0
8,8,0,194,0,0,inf,200,0,0,inf,200,0,0,inf,0
9,9,0,189,0,0,inf,200,0,0,inf,200,0,0,inf,0


In [ ]:
fig = px.line(data_frame=state_df, x='period', y=[col for col in state_df.columns if 'slurry' in col])
fig.show()

In [ ]:
%%time
from src.scenario.scenario_config import default_scenario_config
from src.incoming_wip.incoming_wip_config import IncomingWipConfig

for i in range(7):
    incoming_wip_config = IncomingWipConfig(
        scenario=default_scenario_config,
        rand_seed=i,
        min_wip_cluster=1,
        max_wip_cluster=4,
        max_wip_magnitude=400,
        wafer_slurry_consumption_kg=3,
    )
    incoming_wip = IncomingWipGenerator(incoming_wip_config).generate()
    dp_model = DpModel(default_dp_model_config)
    dp_model.run(incoming_wip)
    state_df = dp_model.get_best_state_df()
    best_end_state = dp_model.get_best_end_state()
    end_cost = best_end_state[1].cost

    plot_incoming_wip_df = pd.DataFrame({'incoming_wip': incoming_wip, 'period': range(len(incoming_wip))})
    fig = px.line(plot_incoming_wip_df, x='period', y='incoming_wip', title=f'incoming wip {i}')
    fig.show()

    fig = px.line(data_frame=state_df, x='period', y=[col for col in state_df.columns if 'slurry' in col], title=f'model states {i}, cost={end_cost}')
    fig.show()

    # Plot model internal states count
    state_count = [len(dp_model.states[i]) for i, j in enumerate(dp_model.states)]
    plot_state_count = pd.DataFrame({'state_count': state_count, 'period': list(range(len(state_count)))})
    fig = px.bar(data_frame=plot_state_count, x='period', y='state_count', title='number of states')
    fig.show()
